<img src="https://raw.githubusercontent.com/Databricks-BR/health/main/image/head_notebook.png" width="1000px">

##### Descrição e Objetivos

Este notebook utiliza o projeto Open Source **MedGemma** para avalição de imagens médicas, como exames de Raio-X. O objetivo é demonstrar a capacidade do modelo em interpretar e fornecer descrições detalhadas de imagens médicas (laudagem), auxiliando profissionais de saúde na análise e de diagnóstico.

##### Referências:

* [GitHub - MedGemma](https://github.com/google-health/medgemma)
* [GitHub - MedGemma - notebook Hugging Face](https://github.com/google-health/medgemma/blob/main/notebooks/quick_start_with_hugging_face.ipynb)
* [Youtube - Google MedGemma - Medical Text and Image Comprehension](https://www.youtube.com/watch?v=Mdr4EVhr6oo)
* [MedGemma - Documentação](https://developers.google.com/health-ai-developer-foundations/medgemma)
* [MedGemma - huggingface/medgemma-4b-pt](https://huggingface.co/google/medgemma-4b-pt)

##### Controle de Versão do Código

| versão | data | autor | e-mail | alterações |
| --- | --- | --- | --- | --- |
| 1.0 | 30-MAI-2025 | Vinicius Fialho | vinicius.fialho@databricks.com | Primeira versão  |
| 1.1 | 30-MAI-2025 | Luis Assunção | luis.assuncao@databricks.com | Revisão  |

##### Descrição do Cluster

Para rodar a demo (em DEV), utilizamos um single cluster com as seguintes configurações.

| Versão do Databricks Runtime | Tipo de Instância |
| --- | --- |
| 14.3 LTS ML | i3.xlarge |

In [0]:
!pip install -U transformers torch torchvision accelerate
!pip install --upgrade "typing_extensions>=4.5" --force-reinstall

dbutils.library.restartPython()

<img src="https://raw.githubusercontent.com/Databricks-BR/health/main/image/logo_hugging_face.png" width="300px">

O projeto MedGemma está disponível no HuggingFace, uma plataforma que hospeda diferentes modelos de machine learning e permite fácil acesso e compartilhamento de modelos treinados.

Para rodar o modelo, será necessário criar uma conta no [HuggingFace](https://huggingface.co/), vincular a conta ao projeto do [MedGemma](https://huggingface.co/google/medgemma-4b-it) clicando em "Acknowledge license" e gerar um token nas configurações de conta (Settings > Access Tokens > Create new token).

Após isso, basta colar o valor do token no widget que aparecerá após rodar a célula a seguir.

In [0]:
# Configurar autenticação no Hugging Face
# Especificação direta do token 
hf_token = "xxxxx-<COLOQUE-AQUI-O-SEU_TOKEN-DO-HUGGINGFACE>-xxxx"

# Login no Hugging Face
from huggingface_hub import login
if hf_token:
    login(token=hf_token)

<img src="https://raw.githubusercontent.com/Databricks-BR/health/main/image/logo_medgemma.png" width="400px">

A coleção MedGemma contém os modelos abertos do Google mais capazes para compreensão de texto e imagem médica, construídos no Gemma 3. Os desenvolvedores podem usar o MedGemma para acelerar a construção de aplicações de IA voltadas para a área da saúde. O MedGemma vem em duas variantes: uma versão multimodal de 4B e uma versão somente texto de 27B.

Para detalhes sobre como usar o modelo e como ele foi treinado, veja o [MedGemma Model Card](https://developers.google.com/health-ai-developer-foundations/medgemma/model-card).

Para mais informações, veja a [Documentação do MedGemma](https://developers.google.com/health-ai-developer-foundations/medgemma).

In [0]:
from transformers import AutoModelForImageTextToText, pipeline
from PIL import Image
import requests
import torch
from IPython.display import Image as IPImage, display, Markdown

pipe = pipeline(
    "image-text-to-text",
    model="google/medgemma-4b-it",
    torch_dtype=torch.bfloat16,
    device="cpu" 
)

In [0]:
# IMAGEM 01
# Atribuição da imagem: Stillwaterising, CC0, via Wikimedia Commons
image_url = (
    "https://upload.wikimedia.org/wikipedia/commons/c/c8/Chest_Xray_PA_3-8-2010.png"
)
image = Image.open(
    requests.get(image_url, headers={"User-Agent": "example"}, stream=True).raw
)
prompt = "Describe this X-ray"

messages = [
    {
        "role": "system",
        "content": [{"type": "text", "text": "You are an expert radiologist."}],
    },
    {
        "role": "user",
        "content": [
            {"type": "text", "text": prompt},
            {"type": "image", "image": image},
        ],
    },
]

# Gerar resposta
output = pipe(text=messages, max_new_tokens=450)
analysis_result = output[0]["generated_text"][-1]["content"]

display(Markdown(f"\n\n---\n\n**[ User ]**\n\n {prompt}"))
display(IPImage(url=image_url, height=300))
display(Markdown(f"\n\n---\n\n**[ MedGemma ]**\n\n {analysis_result}"))

In [0]:
# IMAGEM 02
# Atribuição da imagem: Raio-X do braço de um paciente
image_url = (
    "https://raw.githubusercontent.com/Databricks-BR/health/main/image/wolverine.jpg"
)
image = Image.open(
    requests.get(image_url, headers={"User-Agent": "example"}, stream=True).raw
)
prompt = "Describe this X-ray"

messages = [
    {
        "role": "system",
        "content": [{"type": "text", "text": "You are an expert radiologist."}],
    },
    {
        "role": "user",
        "content": [
            {"type": "text", "text": prompt},
            {"type": "image", "image": image},
        ],
    },
]

# Gerar resposta
output = pipe(text=messages, max_new_tokens=450)
analysis_result = output[0]["generated_text"][-1]["content"]

display(Markdown(f"\n\n---\n\n**[ User ]**\n\n {prompt}"))
display(IPImage(url=image_url, height=300))
display(Markdown(f"\n\n---\n\n**[ MedGemma ]**\n\n {analysis_result}"))